# Throughput
This notebook creates a throughput report for test results. It ties into the **Test Monitor Service** for retrieving filtered test results, the **Notebook Execution Service** for running outside of Jupyterhub, and the **Test Monitor Reports page** at #testmonitor/reports for displaying results.

The parameters and output use a schema recognized by the Test Monitor Reports page, which can be implemented by various report types. The Throughput notebook produces data that is best shown in a bar graph.

### Imports
Import Python modules for executing the notebook. Pandas is used for building and handling dataframes. Scrapbook is used for recording data for the Notebook Execution Service. The SystemLink Test Monitor Client provides access to test result data for processing.

In [1]:
import copy
import datetime
import pandas as pd
import scrapbook as sb
from dateutil import tz

import systemlink.clients.nitestmonitor as testmon

### Parameters
- `results_filter`: Dynamic Linq query filter for test results from the Test Monitor Service  
  Options: Any valid Test Monitor Results Dynamic Linq filter  
  Default: `'startedWithin <= "30.0:0:0"'`
- `group_by`: The dimension along which to reduce; what each bar in the output graph represents  
  Options: Day, System, Test Program, Operator, Part Number  
  Default: Day

Parameters are also listed in the metadata for the parameters cell, along with their default values. The Notebook Execution services uses that metadata to pass parameters from the Test Monitor Reports page to this notebook. Available `group_by` options are listed in the metadata as well; the Test Monitor Reports page uses these to validate inputs sent to the notebook.

To see the metadata, select the code cell and click the wrench icon in the far left panel.

In [2]:
results_filter = 'startedWithin <= "30.0:0:0"'
products_filter = ''

### Create Test Monitor client
Establish a connection to SystemLink over HTTP.

In [3]:
results_api = testmon.ResultsApi()

### Query for results
Query the Test Monitor Service for results matching the `results_filter` parameter.

In [4]:
results_query = testmon.ResultsAdvancedQuery(
    results_filter, product_filter=products_filter, order_by=testmon.ResultField.STARTED_AT
)

results = []

response = await results_api.query_results_v2(post_body=results_query)
while response.continuation_token:
    results = results + response.results
    results_query.continuation_token = response.continuation_token
    response = await results_api.query_results_v2(post_body=results_query)

results_list = [result.to_dict() for result in results]

### Create pandas dataframe
Put the data into a dataframe whose columns are test result id, start time, and group name.

In [8]:
df_results = pd.DataFrame.from_records(results_list)
df_results = df_results.join(df_results['status'].apply(pd.Series))
df_results['started_at'] = df_results['started_at'].apply(lambda t: t.tz_localize(None))
df_results['updated_at'] = df_results['updated_at'].apply(lambda t: t.tz_localize(None))
df_results.pop('status')
df_results.pop('status_type_summary')

,started_at,updated_at,program_name,id,system_id,host_name,operator,part_number,serial_number,total_time_in_seconds,keywords,properties,file_ids,workspace,status_type,status_name,Location
0,2021-01-27 22:02:29.991441,2021-01-27 22:02:30.613514,Program Name,6011e2f526b41e3160c25e20,,,John Doe,B3F2H-STL,123456,37.450061,"[keyword1, keyword2]","{'Location': 'Lab-A', 'nitmBatchSerialNumber':...",[],d725064a-b04a-49c1-a47d-49487df244e7,PASSED,,Lab-A
1,2021-02-02 15:44:45.241747,2021-02-03 02:41:32.340000,Battery Cycle Test.seq,6019736d26b41e3160c25e26,Precision_Tower_5810--SN-8GJJ942--MAC-34-17-EB...,JPREWITT5,administrator,HR-3UTG-AMZN,43231,14.203353,[],"{'nitmSource': 'niteststand', 'nitmTestStandSt...","[6019737c26b41e2820403887, 6019738226b41e28204...",d725064a-b04a-49c1-a47d-49487df244e7,PASSED,None,Austin


In [6]:
df_dict = {
    'columns': pd.io.json.build_table_schema(df_results, index=False)['fields'],
    'values': df_results.values.tolist(),
}

results_graph = {
    'type': 'data_frame',
    'id': 'results_graph',
    'data': df_dict
}

result = [results_graph]

### Record results with Scrapbook

In [7]:
sb.glue('result', result)